## Agents
The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [1]:
%load_ext dotenv
%dotenv

## Goal

We're going to build an agent which serves as a coding assistant. It'll have the design pattern knowledge provided via vector store and also will be able to write and review the code.

### Tools
Tools are functions that an agent can invoke. There are two important design considerations around tools:

- Giving the agent access to the right tools
- Describing the tools in a way that is most helpful to the agent

Without thinking through both, you won't be able to build a working agent. If you don't give the agent access to a correct set of tools, it will never be able to accomplish the objectives you give it. If you don't describe the tools well, the agent won't know how to use them properly.

In [2]:
from langchain.pydantic_v1 import BaseModel, Field
# Design patterns tool

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import tool
from langchain.vectorstores import Chroma

loader = CSVLoader(file_path='../data/design_patterns.csv', source_column='content', metadata_columns=['page_title'])
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
split_docs = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(split_docs, embedding=embeddings)

template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)
vs_retriever_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
vector_store_retriever = db.as_retriever(search_kwargs={"k": 3})

vs_retriever_chain = {'question': RunnablePassthrough(), 'context': vector_store_retriever} | QA_CHAIN_PROMPT | vs_retriever_llm | StrOutputParser()

class DesignPatternsInput(BaseModel):
    design_pattern: str = Field(description="name of the design pattern to learn about")
    
@tool("design_patterns_knowledge", args_schema=DesignPatternsInput)
def design_patterns_knowledge(design_pattern: str) -> str:
    """Source of design patterns knowledge. Use it to gain knowledge about certain design patterns."""
    return vs_retriever_chain.invoke(design_pattern)

In [3]:
from langchain.pydantic_v1 import BaseModel, Field

# Code review tool

review_code_prompt = """Write code review of the given code. Consider the following aspects:
- Are there any creational design patterns applicable?
- Are there any structural design patterns applicable?
- Are there any behavioral design patterns applicable?

Code: {code}

Your review: """

REVIEW_CODE_TEMPLATE = PromptTemplate(
    input_variables=["code"],
    template=review_code_prompt,
)
review_code_llm = ChatOpenAI(temperature=0, model_name="gpt-4")

review_code_chain = REVIEW_CODE_TEMPLATE | review_code_llm | StrOutputParser()

class ReviewCodeInput(BaseModel):
    code: str = Field(description="code to review")

@tool("review_code", args_schema=ReviewCodeInput)
def review_code(code: str) -> str:
    """Use this tool to perform code review."""
    return review_code_chain.invoke({'code': code})

In [4]:
from langchain.pydantic_v1 import BaseModel, Field

# Code writing tool

write_code_prompt = """Write python code based on user description. Try to add descriptive comments to explain what the code does.

Return only python code in Markdown format, e.g.:

```python
....
```

Code description: {description}
Code to refactor: {code_to_refactor}
"""

write_code_template = PromptTemplate(
    input_variables=["description", "code_to_refactor"],
    template=write_code_prompt,
)

def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

write_code_llm = ChatOpenAI(temperature=0, model_name="gpt-4")

write_code_chain = write_code_template | write_code_llm | StrOutputParser() | _sanitize_output

class WriteCodeInput(BaseModel):
    description: str = Field(description="code description to generate")
    code_to_refactor: str = Field(description="optional code to refactor", default="")

@tool("write_code", args_schema=WriteCodeInput)
def write_code(description: str, code_to_refactor: str = "") -> str:
    """Generate python code based on description and optionally, the existing code to refactor."""
    return write_code_chain.invoke({'description': description, 'code_to_refactor': code_to_refactor})

## Agent executor

The agent executor is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats.

In [7]:
from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser

tools = [design_patterns_knowledge, write_code, review_code]


prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent_llm = ChatOpenAI(temperature=0, model_name="gpt-4")
chat_model_with_stop = agent_llm.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
agent_executor.invoke({"input": """
Review the following code. Apply the review suggestions to refactor the code. Return the refactored code.

```python
class Square:
    def __init__(self, side_length):
        self.side_length = side_length

    def area(self):
        return self.side_length * self.side_length

    def perimeter(self):
        return 4 * self.side_length

class Circle:
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14 * self.radius * self.radius

    def circumference(self):
        return 2 * 3.14 * self.radius

def calculate_area(objects):
    for obj in objects:
        if isinstance(obj, Square):
            print("Area of square:", obj.area())
        elif isinstance(obj, Circle):
            print("Area of circle:", obj.area())

def calculate_perimeter(objects):
    for obj in objects:
        if isinstance(obj, Square):
            print("Perimeter of square:", obj.perimeter())
        elif isinstance(obj, Circle):
            print("Circumference of circle:", obj.circumference())

# Usage
shapes = [Square(5), Circle(3)]
calculate_area(shapes)
calculate_perimeter(shapes)
```
"""})



> Entering new AgentExecutor chain...
Thought: The code provided is a simple implementation of two geometric shapes: Square and Circle. Each class has methods to calculate area and perimeter (or circumference for Circle). The functions calculate_area and calculate_perimeter are used to calculate and print the area and perimeter of a list of shapes. However, the code could be improved by using polymorphism, which is a principle of object-oriented programming. This would eliminate the need for the isinstance checks in the calculate_area and calculate_perimeter functions. I will use the review_code tool to provide a review of the code. 

Action:
```
{
  "action": "review_code",
  "action_input": {
    "code": "class Square:\n    def __init__(self, side_length):\n        self.side_length = side_length\n\n    def area(self):\n        return self.side_length * self.side_length\n\n    def perimeter(self):\n        return 4 * self.side_length\n\nclass Circle:\n    def __init__(self, radius):

{'input': '\nReview the following code. Apply the review suggestions to refactor the code. Return the refactored code.\n\n```python\nclass Square:\n    def __init__(self, side_length):\n        self.side_length = side_length\n\n    def area(self):\n        return self.side_length * self.side_length\n\n    def perimeter(self):\n        return 4 * self.side_length\n\nclass Circle:\n    def __init__(self, radius):\n        self.radius = radius\n\n    def area(self):\n        return 3.14 * self.radius * self.radius\n\n    def circumference(self):\n        return 2 * 3.14 * self.radius\n\ndef calculate_area(objects):\n    for obj in objects:\n        if isinstance(obj, Square):\n            print("Area of square:", obj.area())\n        elif isinstance(obj, Circle):\n            print("Area of circle:", obj.area())\n\ndef calculate_perimeter(objects):\n    for obj in objects:\n        if isinstance(obj, Square):\n            print("Perimeter of square:", obj.perimeter())\n        elif isinst